In [54]:
import pandas as pd
import plotly.express as px
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import datetime
import time

## Finding the ISS

In [83]:
# Reading in json data from API
url = 'http://api.open-notify.org/iss-now.json'
currentposition_df = pd.read_json(url)

In [84]:
# Wranging the data into the needed format
currentposition_df['latitude'] = currentposition_df.loc['latitude', 'iss_position']
currentposition_df['longitude'] = currentposition_df.loc['longitude', 'iss_position']
currentposition_df.reset_index(inplace=True)
currentposition_df = currentposition_df.drop(['index', 'message', 'iss_position'], axis = 1)
currentposition_df.drop_duplicates(subset=['latitude', 'longitude'], inplace=True)
currentposition_df

,timestamp,latitude,longitude
0,2022-04-03 19:12:47,26.8157,21.1313


In [85]:
# Plotting the ISS's current position
fig = px.scatter_geo(currentposition_df, lat='latitude', lon='longitude')
fig.show()

In [81]:
# Recording the trajectory using a loop
record_data = True # needs to be manually switched on (True)
if record_data == True:
    starttime=time.time() # starting a timer (new position will be added every 60 seconds)
    space_station_data_df = pd.DataFrame(columns=['timestamp',	'latitude',	'longitude']) # creating an empty df for our observations (though a list would be more efficient)

    while True: 
        url = 'http://api.open-notify.org/iss-now.json'
        furtherposition_df = pd.read_json(url)
        furtherposition_df['latitude'] = furtherposition_df.loc['latitude', 'iss_position']
        furtherposition_df['longitude'] = furtherposition_df.loc['longitude', 'iss_position']
        furtherposition_df.reset_index(inplace=True)
        furtherposition_df = furtherposition_df.drop(['index', 'message', 'iss_position'], axis = 1)
        furtherposition_df.drop_duplicates(subset=['latitude', 'longitude'], inplace=True)

        space_station_data_df = pd.concat([space_station_data_df, furtherposition_df]) # add newest obversation to the previous ones
        print(space_station_data_df.iloc[-1]) # print last line to make the process visible

        if len(space_station_data) > 18000: # break if the process goes on for longer than 5 hours (18000 seconds)
            break

        # let it sleep 60 seconds
        time.sleep(6.0 - ((time.time() - starttime) % 6.0)) # wait for 60 seconds before entering the while loop anew

timestamp    2022-04-03 19:11:19
latitude                 30.8568
longitude                16.8654
Name: 0, dtype: object
timestamp    2022-04-03 19:11:25
latitude                  30.588
longitude                17.1664
Name: 0, dtype: object
timestamp    2022-04-03 19:11:31
latitude                 30.3184
longitude                17.4656
Name: 0, dtype: object
timestamp    2022-04-03 19:11:37
latitude                 30.0481
longitude                17.7629
Name: 0, dtype: object
timestamp    2022-04-03 19:11:43
latitude                 29.7769
longitude                18.0585
Name: 0, dtype: object
timestamp    2022-04-03 19:11:49
latitude                  29.505
longitude                18.3523
Name: 0, dtype: object
timestamp    2022-04-03 19:11:55
latitude                 29.2323
longitude                18.6445
Name: 0, dtype: object


KeyboardInterrupt: 

In [82]:
# saving df to file after the process of collecting data has finished
space_station_data_df.to_csv('ISS_locations.csv', index=None)